# **Enunciado**

## José Daniel Sarmiento Blanco 2192232

Design a heat exchanger to be used as a crude oil cooler. 120 kg/s of crude oil enters to the heat exchanger at 102°C and leaves at 65°C. The coolant to be used is city water entering the tube side at 21°C with a flow rate of 65 kg/s. Pressure drop for the crude oil is limited to 150 kPa.

In [147]:
caudal_crudo             = 120 # kg/s
temperatura_entrada_crudo   = 102 # ºC
temperatura_salida_crudo    = 65  # ºC

temperatura_entrada_agua    = 21  # ºC
caudal_agua                 = 65  # kg/s

presion_limite_crudo        = 150_000 # Pa

In [148]:
from pyfluids import Fluid, FluidsList, Input, Mixture
from core import ThermoFormules
import numpy as np

<p align="center">
  <img src="./diagrama.jpeg" height ="600px">
</p>

## 0. Establecer valores pre-diseño

In [149]:
k_material                            = 60    # W/ m * k
diametro_interior                     = 16e-3 # m
diametro_exterior                     = 19e-3 # m

longitud                              = 4 #m               

In [150]:
water = Fluid(FluidsList.Water)
water.name

Water

### 0.1 Suposición

[presión agua](https://blog.verti.es/hogar/presion-agua-casa/#:~:text=Lo%20habitual%20es%20que%20la,tengamos%20una%20presión%20u%20otra.)


- Considere como material el acero al carbono, k = 60 W/m*k.
- Diámetro de tubo de 3/4 in ( 19 mm exterior y 16 mm interior ).
- Longitud de 4m

In [151]:
presion_agua  = (2) * 101_325 # Pa
presion_crudo = 413_000       # Pa

### Composición del crudo

In [152]:
components = [FluidsList.nHexane, FluidsList.nOctane, FluidsList.nDecane]
fractions = [30, 40, 30]
mixture = dict(zip(components, fractions))
mixture

{nHexane: 30, nOctane: 40, nDecane: 30}

In [153]:
crudo = Mixture(
    components,
    fractions=fractions,
)

In [154]:
temperatura_propiedades_crudo = (temperatura_entrada_crudo + temperatura_salida_crudo) / 2
print(temperatura_propiedades_crudo )#+ 273.15)

cp_propiedades_crudo =  0.83 * 1000  # J/kg k

83.5


## 1. Termodinámica

$$ \dot{Q} = (\dot{m} *C_{p})_{crudo} * (T_{c1} -T_{c2})$$

In [155]:
temperatura_propiedades_crudo = (temperatura_entrada_crudo + temperatura_salida_crudo) / 2
print(temperatura_propiedades_crudo) #+ 273.15)

cp_propiedades_crudo =  0.83 * 1000  # J/kg k

83.5


In [156]:
calor = caudal_crudo * cp_propiedades_crudo * (temperatura_entrada_crudo - temperatura_salida_crudo ) 
calor

3685200.0

In [157]:
estado_agua_propiedades = water.with_state(
    Input.pressure(presion_agua),
    Input.temperature(34),
)
estado_agua_propiedades.as_dict()

{'compressibility': 0.0014376012709739205,
 'conductivity': 0.6203366408703609,
 'critical_pressure': 22064000.0,
 'critical_temperature': 373.946,
 'density': 994.417937160409,
 'dynamic_viscosity': 0.0007337301352239351,
 'enthalpy': 142631.9153801642,
 'entropy': 491.47808690964223,
 'fraction': 100,
 'freezing_temperature': None,
 'internal_energy': 142428.1278251999,
 'kinematic_viscosity': 7.378488538924831e-07,
 'max_pressure': 1000000000.0,
 'max_temperature': 1726.85,
 'min_pressure': 611.6548008968684,
 'min_temperature': 0.010000000000047748,
 'molar_mass': 0.018015268,
 'name': Water,
 'phase': Liquid,
 'prandtl': 4.942945501085879,
 'pressure': 202650,
 'quality': None,
 'sound_speed': 1518.0155752419128,
 'specific_heat': 4179.043576032273,
 'specific_volume': 0.0010056133971753675,
 'surface_tension': None,
 'temperature': 34.0,
 'triple_pressure': 611.6548008968684,
 'triple_temperature': 0.010000000000047748,
 'units_system': SIWithCelsiusAndPercents}

In [158]:
cp_agua                         = estado_agua_propiedades.specific_heat # J /kg k
viscosidad_dinamica_agua        = estado_agua_propiedades.dynamic_viscosity # Pa * s
densidad_agua                   = estado_agua_propiedades.density # kg / m3
k_agua                          = estado_agua_propiedades.conductivity # W / m k
prandl_agua                     = estado_agua_propiedades.prandtl

$$ \dot{Q} = (\dot{m} *C_{p})_{agua} * (T_{w1} - T_{w2})$$

In [159]:
temperatura_salida_agua = temperatura_entrada_agua + calor / (caudal_agua * cp_agua)
print("la temperatura de lsa salida del agua del agua es: ", temperatura_salida_agua)

la temperatura de lsa salida del agua del agua es:  34.566593308704654


## 2. Transferencia de calor

<p align="center">
  <img src="./resistencia.jpeg" height ="200px">
</p>

### 2.1 Calculo de una U estimada

Asuma los coeficiente de trasferencia de calor

$$ \frac{1}{U_{ensuciamiento}} = \frac{r_{0}}{r_{i}} \frac{1}{h_{i}} + \frac{r_{0}}{r_{1}}*R_{f_{}i} + r_{0} \frac{ln(\frac{r_{0}}{r_{i}} )}{k} + R_{f_{}o} + \frac{1}{h_{0}} $$



$$ \frac{1}{U_{limpio}} = \frac{r_{0}}{r_{i}} \frac{1}{h_{i}} + r_{0} \frac{ln(\frac{r_{0}}{r_{i}} )}{k} + \frac{1}{h_{0}}$$

En este caso
- $ r_{0} $ -> radio exterior
- $ r_{i} $ -> radio interior

- $ h_{0} $ -> coeficiente de convección en la carcasa
- $ h_{i} $ -> coeficiente de convección en el tubo

In [215]:
h_agua    = 4500     # kj/kg * k
h_crudo   = 800      # kj/kg * k

R_f_crudo = 0.000352 # m2 k/w
R_f_agua  = 0.000176 # m2 k/w

In [228]:
U_ensuciamiento = (
    (diametro_exterior / diametro_interior) * 1 / h_crudo
    + (diametro_exterior / diametro_interior) * R_f_crudo
    + (diametro_exterior / 2) * np.log(diametro_exterior/diametro_interior) / k_material
    + R_f_agua
    + 1 / h_agua
) ** -1 
U_ensuciamiento

429.5889075234658

In [217]:
U_no_ensuciamiento = (
    (diametro_exterior / diametro_interior) * 1 / h_crudo
    + (diametro_exterior / 2) * np.log(diametro_exterior / diametro_interior) / k_material
    + 1 / h_agua
) ** -1
U_no_ensuciamiento

576.7655158192335

### 2.2 Calculo de la diferencia media logarítmica

In [234]:
diferencia_temperatura_1 = temperatura_entrada_crudo - temperatura_salida_agua
diferencia_temperatura_2 = temperatura_salida_crudo - temperatura_entrada_agua
print(f"diferencia de temperatura sección 1: {diferencia_temperatura_1:.2f} ºC" )
print(f"diferencia de temperatura sección 2: {diferencia_temperatura_2:.2f} ºC" )

diferencia_log_temperatura = (diferencia_temperatura_2 - diferencia_temperatura_1)/ np.log(diferencia_temperatura_2/diferencia_temperatura_1)
print(f"diferencia media logarítmica: {diferencia_log_temperatura:.2f} ºC")

diferencia de temperatura sección 1: 67.43 ºC
diferencia de temperatura sección 2: 44.00 ºC
diferencia media logarítmica: 54.89 ºC


Para el calculo de las el factor de correction se uso las tables del metodo de lmtd del moodle 

In [235]:
R = (temperatura_entrada_crudo - temperatura_salida_crudo) / (temperatura_salida_agua- temperatura_entrada_agua)
P = 1 / R
print({"R":R, "P":P})

{'R': 2.727287474318251, 'P': 0.36666468401904473}


Teniendo en cuenta las gráficas de moodle

In [221]:
F = 0.86

In [222]:
diferencia_log_temperatura_corregida =  F * diferencia_log_temperatura

### Calculo de la relación de area

In [223]:
A_f = calor / (U_ensuciamiento * diferencia_log_temperatura_corregida)
A_c = calor / (U_no_ensuciamiento * diferencia_log_temperatura_corregida)

print(A_f, A_c)

181.7406452013175 135.3648286578791


In [224]:
area_ratio =  A_f / A_c - 1
area_ratio

0.34259871639662465

Se toma como referencia las siguientes dimisiones y parámetros de diseño

In [170]:
Cl         = 1
L          = 4   # m
CTP        = 0.93 # One tube pass
PitchRatio = 1.25

In [171]:
diametro_carcasa = 0.637 * (Cl / CTP * A_f * pow(PitchRatio, 2) * diametro_exterior/L)**(1/2) 
print(diametro_carcasa)
diametro_carcasa = 0.7
print(diametro_carcasa)

0.7579967962632624
0.7


In [172]:
numeros_tubos = 0.785 * (CTP/Cl) * (diametro_carcasa)**2 /((PitchRatio * diametro_exterior)**2)
print(numeros_tubos)

634.1930193905818


Se supone un 60 %  del diámetro de la carcasa

In [173]:
B = 0.6 * diametro_carcasa
print('espacio entre baffle', B)
B = 0.5
print('espacio entre baffle', B)

espacio entre baffle 0.42
espacio entre baffle 0.5


## Corrección método de kern

In [174]:
numeros_tubos = 640

In [175]:
T_b = (temperatura_entrada_agua + temperatura_salida_agua) / 2
print(T_b)

27.783296654352327


In [176]:
Pitch_Size              = 0.0254 # m

In [177]:
D_e = 4*(Pitch_Size**2 - np.pi*diametro_exterior**2/4)/(np.pi*diametro_exterior)
D_e

0.024233853930596717

In [178]:
C = Pitch_Size - diametro_exterior
print(C)

0.0063999999999999994


In [179]:
A_s = (0.39 * C * B) / Pitch_Size

print(A_s)

0.04913385826771654


In [180]:
G_s = caudal_crudo / A_s

print(G_s)

2442.307692307692


In [181]:
state_agua_kern = water.with_state(
    Input.pressure(presion_agua),
    Input.temperature(T_b),
)
state_agua_kern.as_dict()

{'compressibility': 0.0014644647194993476,
 'conductivity': 0.6110301728439101,
 'critical_pressure': 22064000.0,
 'critical_temperature': 373.946,
 'density': 996.3427400551607,
 'dynamic_viscosity': 0.0008363303275519409,
 'enthalpy': 116649.56106539103,
 'entropy': 406.01851083797783,
 'fraction': 100,
 'freezing_temperature': None,
 'internal_energy': 116446.16720124884,
 'kinematic_viscosity': 8.394002323995847e-07,
 'max_pressure': 1000000000.0,
 'max_temperature': 1726.85,
 'min_pressure': 611.6548008968684,
 'min_temperature': 0.010000000000047748,
 'molar_mass': 0.018015268,
 'name': Water,
 'phase': Liquid,
 'prandtl': 5.721351230984074,
 'pressure': 202650,
 'quality': None,
 'sound_speed': 1504.0295438369426,
 'specific_heat': 4180.068707781974,
 'specific_volume': 0.0010036706845925698,
 'surface_tension': None,
 'temperature': 27.78329665435234,
 'triple_pressure': 611.6548008968684,
 'triple_temperature': 0.010000000000047748,
 'units_system': SIWithCelsiusAndPercents}

In [182]:
densidad_agua_kern            = state_agua_kern.density
cp_agua_kern                  = state_agua_kern.specific_heat
viscosidad_dinamica_agua_kern = state_agua_kern.dynamic_viscosity
k_agua_kern                   = state_agua_kern.conductivity
prandtl_agua_kern             = state_agua_kern.prandtl

print(densidad_agua_kern, cp_agua_kern, viscosidad_dinamica_agua_kern, k_agua_kern, prandtl_agua_kern)

996.3427400551607 4180.068707781974 0.0008363303275519409 0.6110301728439101 5.721351230984074


In [183]:
crudo = Mixture(
    components,
    fractions=fractions,
)
estado_crudo =  crudo.with_state(
    Input.pressure(101_000),
    Input.temperature(T_b),
)
estado_crudo.as_dict()

{'compressibility': 0.006436755777148306,
 'conductivity': 0.14217831788022434,
 'critical_pressure': 2981833.4687300003,
 'critical_temperature': 298.33744539093857,
 'density': 689.7858668774503,
 'dynamic_viscosity': None,
 'enthalpy': -232830.54755720525,
 'entropy': -574.4798416742174,
 'fluids': [nHexane, nOctane, nDecane],
 'fractions': [30, 40, 30],
 'freezing_temperature': None,
 'internal_energy': -232976.9698098167,
 'kinematic_viscosity': None,
 'max_pressure': 605929364.2590215,
 'max_temperature': 394.3155003716736,
 'min_pressure': 1.5799947781094636,
 'min_temperature': -65.24557909004545,
 'molar_mass': 0.1099927925291343,
 'phase': Liquid,
 'prandtl': None,
 'pressure': 101000,
 'quality': None,
 'sound_speed': 1141.6288890694889,
 'specific_heat': 2235.802285290991,
 'specific_volume': 0.0014497252669539884,
 'surface_tension': None,
 'temperature': 27.78329665435234,
 'triple_pressure': 1.5799947781094636,
 'triple_temperature': -65.24557909004545,
 'units_system': 

In [184]:
densidad_crudo_kern            = estado_crudo.density
cp_crudo_kern                  = estado_crudo.specific_heat
viscosidad_dinamica_crudo_kern = 0.001
k_crudo_kern                   = estado_crudo.conductivity
prandtl_crudo_kern             = estado_crudo.prandtl

print(densidad_crudo_kern, cp_crudo_kern, viscosidad_dinamica_crudo_kern, k_crudo_kern, prandtl_crudo_kern)

689.7858668774503 2235.802285290991 0.001 0.14217831788022434 None


In [185]:
Re = G_s * D_e / viscosidad_dinamica_crudo_kern

print('Reynolds: ', Re)

Reynolds:  59186.52786895735


In [186]:
T_w = 1/2 * (
    (temperatura_entrada_agua + temperatura_salida_agua)/2 +
    (temperatura_entrada_crudo + temperatura_salida_crudo)/2
)
T_w

55.64164832717616

In [187]:
state_average = water.with_state(
    Input.pressure(presion_agua),
    Input.temperature(T_w),
)
state_average.as_dict()

{'compressibility': 0.0013552312320620485,
 'conductivity': 0.6467357483177187,
 'critical_pressure': 22064000.0,
 'critical_temperature': 373.946,
 'density': 985.4253674034841,
 'dynamic_viscosity': 0.0004985467955942873,
 'enthalpy': 233099.21479641055,
 'entropy': 776.1009136555505,
 'fraction': 100,
 'freezing_temperature': None,
 'internal_energy': 232893.56756356746,
 'kinematic_viscosity': 5.059203995406752e-07,
 'max_pressure': 1000000000.0,
 'max_temperature': 1726.85,
 'min_pressure': 611.6548008968684,
 'min_temperature': 0.010000000000047748,
 'molar_mass': 0.018015268,
 'name': Water,
 'phase': Liquid,
 'prandtl': 3.2245057186580506,
 'pressure': 202650,
 'quality': None,
 'sound_speed': 1548.1033156097012,
 'specific_heat': 4182.963640203917,
 'specific_volume': 0.0010147901942436483,
 'surface_tension': None,
 'temperature': 55.64164832717614,
 'triple_pressure': 611.6548008968684,
 'triple_temperature': 0.010000000000047748,
 'units_system': SIWithCelsiusAndPercents}

In [188]:
viscodiad_dinamica_agua_kern = state_average.dynamic_viscosity
viscodiad_dinamica_agua_kern

0.0004985467955942873

In [189]:
h_0 = 0.36 * (D_e * G_s / viscosidad_dinamica_crudo_kern)**0.55  * (cp_crudo_kern * viscosidad_dinamica_crudo_kern / k_crudo_kern)**(1/3) * (viscosidad_dinamica_crudo_kern / viscodiad_dinamica_agua_kern)**0.14 * k_crudo_kern / D_e
h_0

2458.2373104527564

In [190]:
A_tp = np.pi * diametro_interior**2/4 * numeros_tubos/2 
A_tp

0.06433981754551896

In [191]:
u_m = caudal_agua / (densidad_agua_kern * A_tp)
print(u_m)

1.0139692182428435


In [192]:
Re = densidad_agua_kern * u_m * diametro_interior / viscosidad_dinamica_agua_kern
Re

19327.499404554037

### Como el fluido es turbulento se usa correlación de  Grielinki's

In [193]:
f = (1.58*np.log(Re) - 3.28)**-2
f

0.0065953698516935284

In [194]:
Nu_b = (f/2)*(Re - 1000)*prandtl_agua_kern/(1 + 12.7*(f/2)**(1/2)*(prandtl_agua_kern**(2/3) - 1))
Nu_b

132.8089134819997

In [195]:
h_i = Nu_b * k_agua_kern/diametro_interior
print(h_i)

5071.890835007386


In [201]:
U_f = (
    diametro_exterior / (diametro_interior * h_i) + 
    diametro_exterior / diametro_interior * R_f_crudo + 
    diametro_exterior * np.log(diametro_exterior / diametro_interior) / (2 * k_material) + 
    1/h_0
) ** -1
U_f

920.6926723779326

In [200]:
U_c = (
    diametro_exterior / (diametro_interior * h_i) + 
    diametro_exterior * np.log(diametro_exterior / diametro_interior) / (2 * k_material) + 
    1 / h_0
) ** -1
U_c

1496.695081669341

In [198]:
f = np.e**(0.576 - 0.19*np.log(Re))
print(f)

N_b = longitud / B - 1 
print(N_b)

sigma = (viscosidad_dinamica_crudo_kern / viscosidad_dinamica_agua_kern)**0.14
print(sigma)

diferencia_presion = f * G_s ** 2 * (N_b + 1) * 0.39 / (sigma * 2 * densidad_agua_kern * D_e)
diferencia_presion

0.2727597254007179
7.0
1.025338114643251


102519.68178099226

In [199]:
area = calor /(U_f * diferencia_log_temperatura)
area

58.851783401904335

### da una diferencia de presión inferior a 150kpa (85090) por ende cumple con los requerimientos


![](./matches.png)

### Resultados de la evaluación de diseño

- Material = Acero al carbono k = 60 W/mk
- Longitud del intercambiador = 4 m
- Numero de tubos = 640
- Coeficientes de convección
    - Agua : 5071.76 W/mk
    - Crudo : 2458.236773857342


    

- Diámetro de la carcasa: 0.7 m
- La distribución de los tubos en cuadrada
- Un Pitch_Size de 0.0254 m

- Temperaturas de trabajo
    - Agua :
        - Entrada : 21 º C
        - Salida  : 34.5 º C
    - Crudo :
        - Entrada : 102 ºC
        - Salida  : 65  ºC


## Referencias
- Abdulagatov, I.M., Abdulagatova, Z.Z., Grigor’ev, B.A. et al. Thermal Diffusivity, Heat Capacity, and Thermal Conductivity of Oil Reservoir Rock at High Temperatures. Int J Thermophys 42, 135 (2021). https://doi.org/10.1007/s10765-021-02878-x
- Seguros, V. (2021) Presión del Agua en Casa: Problemas habituales: Blog Verti, Verti Blog. Available at: https://blog.verti.es/hogar/presion-agua-casa/#:~:text=Lo%20habitual%20es%20que%20la,tengamos%20una%20presión%20u%20otra. (Accessed: 13 October 2023). 
- Bell, I.H. et al. (2014) ‘Pure and pseudo-pure fluid thermophysical property evaluation and the open-source Thermophysical Property Library coolprop’, Industrial &amp;amp; Engineering Chemistry Research, 53(6), pp. 2498–2508. doi:10.1021/ie4033999. 